In [2]:
!nvidia-smi

Mon Jul 26 17:23:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# YFINANCE an open source API
! pip install yfinance

     |████████████████████████████████| 6.3 MB 12.1 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=cf72addcd389a4f20c5e669ce45752dbc882cf1b8a2feafca9967e01944ab637
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
# google ticker for google stock price # interval - frequncy at which we fetch our data 
# interval - "1d", i.e., one row of data representing each day
data = yf.download("GOOGL", start="2018-01-01", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [6]:
data.shape

(897, 6)

In [7]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [8]:
# sort the data points based on indexes just for confirmation
data.sort_index(inplace=True)

In [9]:
# remove the duplicate index
data = data.loc[~data.index.duplicated(keep='first')]

In [10]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-22,2556.879883,2569.659912,2549.979980,2568.429932,2568.429932,715100
2021-07-23,2608.610107,2667.979980,2596.010010,2660.300049,2660.300049,2072200
2021-07-26,2666.560059,2674.000000,2646.030029,2666.260010,2666.260010,721921


In [11]:
# check missing data
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [12]:
# print statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,897.000000,897.000000,897.000000,897.000000,897.000000,8.970000e+02
mean,1411.217468,1426.168125,1396.906290,1412.245250,1412.245250,1.809893e+06
std,396.108147,399.042806,393.941943,397.006854,397.006854,8.428353e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1132.310059,1144.140015,1117.500000,1131.550049,1131.550049,1.272500e+06
50%,1241.810059,1248.989990,1228.000000,1240.140015,1240.140015,1.584200e+06
75%,1523.109985,1540.890015,1508.209961,1519.439941,1519.439941,2.077400e+06
max,2666.560059,2674.000000,2646.030029,2666.260010,2666.260010,6.658900e+06


## Data Visualization

In [13]:
import plotly.graph_objects as go

# check the trend in Closing Values
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines'))
fig.update_layout(height=500, width=900,
                  xaxis_title='Date', yaxis_title='Close')
fig.show()

In [14]:
# check the trend in Volume Values
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['Volume'], mode='lines'))
fig.update_layout(height=500, width=900,
                  xaxis_title='Date', yaxis_title='Volume')
fig.show()

In [15]:
# check the trend in other values
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Open'] , mode = 'lines' , name='Open'))
fig.add_trace(go.Scatter(x = data.index , y = data['High'] , mode = 'lines' , name='High'))
fig.add_trace(go.Scatter(x = data.index , y = data['Low'] , mode = 'lines' , name='Low'))
fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines' , name='Close'))
fig.add_trace(go.Scatter(x = data.index , y = data['Adj Close'] , mode = 'lines' , name='Adj Close'))
# fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines' , name='Volume'))

fig.show()

## Data Preprocessing

In [16]:
from sklearn.preprocessing import MinMaxScaler # scale the dataset
import pickle # save and retrive any python objects
from tqdm.notebook import tnrange # graphical progress bar to track the pre-progress of our preprocessing

In [17]:
# filter only required data
# column open, close, high and low have very common data so choose only one among them
data = data[['Close', 'Volume']]
data.head(3) 

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


In [18]:
# set up test set size
test_length = data[(data.index >= '2020-09-01')].shape[0]
test_length

226

In [19]:
def Features_and_Targets(data, feature_length):
  # create features and targets

  X = []
  y = []

  for i in tnrange(len(data) - feature_length):
    X.append(data.iloc[i:i+feature_length, :].values)
    y.append(data['Close'].values[i+feature_length])

  X = np.array(X)
  y = np.array(y)

  return X, y

In [20]:
X, y = Features_and_Targets(data, 32)

In [21]:
X.shape, y.shape # record, days, values

((865, 32, 2), (865,))

In [22]:
# split into train and test set
X_train, X_test, y_train, y_test = X[:-test_length], X[-test_length:], y[:-test_length], y[-test_length:]

In [23]:
# check training dataset
X_train.shape, y_train.shape

((639, 32, 2), (639,))

In [24]:
# check testing dataset
X_test.shape, y_test.shape

((226, 32, 2), (226,))

In [25]:
# create a scaler to scale vectors with multiple dimensions 
# sklearn only supports scaling on two dimensions but we need to do it on three dimensions
class MultiDimensionScaler():

  # initialize an empty list of scalers
  def __init__(self):
    self.scalers = []

  # looping over third dimension of our data and at each loop we create a new scaler and fit it over that dimension
  def fit_transform(self , X):
    total_dims = X.shape[2]
    for i in range(total_dims):
        Scaler = MinMaxScaler()
        X[:, :, i] = Scaler.fit_transform(X[:, :, i])
        self.scalers.append(Scaler)
    return X # keep collecting the fitted scalers and returns the transformed data


  # again do over the third dimension but apply fitted scalers on the transformed data
  def transform(self, X):
    for i in range(X.shape[2]):
      X[:, :, i] = self.scalers[i].transform(X[:, :, i])
    return X

In [26]:
# apply the above on our features
Feature_Scaler = MultiDimensionScaler()
X_train = Feature_Scaler.fit_transform(X_train)
X_test = Feature_Scaler.transform(X_test)

In [27]:
# apply the MinMaxScaler on targets
Target_Scaler = MinMaxScaler()
y_train = Target_Scaler.fit_transform(y_train.reshape(-1,1))
y_test = Target_Scaler.transform(y_test.reshape(-1,1))

In [28]:
# create two functions to save and load python objects
def save_object(obj, name: str):
  pickle_out = open(f"{name}.pck", "wb")
  pickle.dump(obj, pickle_out)
  pickle_out.close()

def load_obj(name: str):
  pickle_in = open(f"{name}.pck", "rb")
  data = pickle.load(pickle_in)
  return data

In [29]:
# save the objects for future use
save_object(Feature_Scaler, "Feature_Scaler")
save_object(Target_Scaler, "Target_Scaler")

## Model Building  

In [30]:
# define callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', 
                            save_best_only=True, save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                              patience=5, min_lr=0.00001, verbose=1)

In [31]:
# create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512, return_sequences=True, recurrent_dropout=0.1, input_shape=(32,2)))) # return sequence=True because sequence matters
model.add(LSTM(256, recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64, activation='elu')) # modified version of leaky relu (Relu+Tanh merged)
model.add(Dropout(0.3))
model.add(Dense(32, activation='elu'))
model.add(Dense(1, activation='linear'))

In [32]:
# compile the model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.002) 
# use SGD because of batch size, we need a frequent update so batch size has to be low, 
# and SGD is better wrt Adam with lesser batch size

model.compile(loss='mse', optimizer=optimizer)

In [33]:
history = model.fit(X_train, y_train,
                    epochs=10, batch_size=1,
                    verbose=1, shuffle=False,
                    validation_data=(X_test, y_test),
                    callbacks=[reduce_lr, save_best])

Epoch 1/10
639/639 [==============================] - 94s 137ms/step - loss: 0.0122 - val_loss: 0.6109
Epoch 2/10
639/639 [==============================] - 87s 136ms/step - loss: 0.0151 - val_loss: 0.4144
Epoch 3/10
639/639 [==============================] - 86s 135ms/step - loss: 0.0120 - val_loss: 0.3177
Epoch 4/10
639/639 [==============================] - 86s 135ms/step - loss: 0.0091 - val_loss: 0.2855
Epoch 5/10
639/639 [==============================] - 86s 134ms/step - loss: 0.0078 - val_loss: 0.2359
Epoch 6/10
639/639 [==============================] - 85s 133ms/step - loss: 0.0066 - val_loss: 0.1876
Epoch 7/10
639/639 [==============================] - 87s 136ms/step - loss: 0.0064 - val_loss: 0.1699
Epoch 8/10
639/639 [==============================] - 86s 135ms/step - loss: 0.0055 - val_loss: 0.1654
Epoch 9/10
639/639 [==============================] - 87s 136ms/step - loss: 0.0053 - val_loss: 0.1446
Epoch 10/10
639/639 [==============================] - 86s 134ms/step - l

## Visualizing the predictions on Test Set

In [34]:
pred = model.predict(X_test)

In [35]:
Predictions = Target_Scaler.inverse_transform(pred)
Actual = Target_Scaler.inverse_transform(y_test)

In [36]:
Predictions.shape

(226, 1)

In [37]:
Predictions = np.squeeze(Predictions, axis=1) # removes any unwanted axis with value "1 "
Actual = np.squeeze(Actual, axis=1)

In [43]:
# plot Actual vs Predicted Values
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:], y = Actual, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:], y = Predictions, mode='lines', name='Predictions'))
fig.show()

## Scraping Extra Features for Modeling  
RSI Indicator (Relative Strength Index) is a momentum indicator used in technical analysis that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset.

In [56]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json() # convert data received into json format

In [57]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [58]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'], orient='index')
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [59]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']

In [60]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [61]:
# merge data
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [72]:
data.head()

,Close,Volume,RSI
2018-01-02,1073.209961,1588300,65.4310
2018-01-03,1091.520020,1565900,78.3686
2018-01-04,1095.760010,1302600,80.4830
2018-01-05,1110.290039,1512500,86.2429
2018-01-08,1114.209961,1232200,87.4882


## Re-Training the model

In [74]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]
test_length

225

In [84]:
def Features_and_Targets(data, feature_length):
    X = []
    y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    y = np.array(y)

    return X , y

In [85]:
X , y = Features_and_Targets(data , 32)

In [86]:
X.shape, y.shape # record, days, values

((864, 32, 3), (864,))

In [87]:
# split into train and test set
X_train, X_test, y_train, y_test = X[:-test_length], X[-test_length:], y[:-test_length], y[-test_length:]

In [88]:
# check training dataset
X_train.shape, y_train.shape

((639, 32, 3), (639,))

In [89]:
# check testing dataset
X_test.shape, y_test.shape

((225, 32, 3), (225,))

In [90]:
# create a scaler to scale vectors with multiple dimensions 
# sklearn only supports scaling on two dimensions but we need to do it on three dimensions
class MultiDimensionScaler():

  # initialize an empty list of scalers
  def __init__(self):
    self.scalers = []

  # looping over third dimension of our data and at each loop we create a new scaler and fit it over that dimension
  def fit_transform(self , X):
    total_dims = X.shape[2]
    for i in range(total_dims):
        Scaler = MinMaxScaler()
        X[:, :, i] = Scaler.fit_transform(X[:, :, i])
        self.scalers.append(Scaler)
    return X # keep collecting the fitted scalers and returns the transformed data


  # again do over the third dimension but apply fitted scalers on the transformed data
  def transform(self, X):
    for i in range(X.shape[2]):
      X[:, :, i] = self.scalers[i].transform(X[:, :, i])
    return X

In [91]:
# apply the above on our features
Feature_Scaler = MultiDimensionScaler()
X_train = Feature_Scaler.fit_transform(X_train)
X_test = Feature_Scaler.transform(X_test)

In [92]:
# apply the MinMaxScaler on targets
Target_Scaler = MinMaxScaler()
y_train = Target_Scaler.fit_transform(y_train.reshape(-1,1))
y_test = Target_Scaler.transform(y_test.reshape(-1,1))

In [93]:
# create two functions to save and load python objects
def save_object(obj, name: str):
  pickle_out = open(f"{name}.pck", "wb")
  pickle.dump(obj, pickle_out)
  pickle_out.close()

def load_obj(name: str):
  pickle_in = open(f"{name}.pck", "rb")
  data = pickle.load(pickle_in)
  return data

In [95]:
# # save the objects for future use
save_object(Feature_Scaler, "Feature_Scaler")
save_object(Target_Scaler, "Target_Scaler")

In [94]:
# define callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', 
                            save_best_only=True, save_weights_only=True)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                              patience=5, min_lr=0.00001, verbose=1)

In [96]:
# create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512, return_sequences=True, recurrent_dropout=0.1, input_shape=(32,2)))) # return sequence=True because sequence matters
model.add(LSTM(256, recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64, activation='elu')) # modified version of leaky relu (Relu+Tanh merged)
model.add(Dropout(0.3))
model.add(Dense(32, activation='elu'))
model.add(Dense(1, activation='linear'))



# compile the model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.002) 
model.compile(loss='mse', optimizer=optimizer)

In [97]:
history = model.fit(X_train, y_train,
                    epochs=10, batch_size=1,
                    verbose=1, shuffle=False,
                    validation_data=(X_test, y_test),
                    callbacks=[reduce_lr, save_best])

Epoch 1/10
639/639 [==============================] - 90s 135ms/step - loss: 0.0086 - val_loss: 0.3730
Epoch 2/10
639/639 [==============================] - 86s 134ms/step - loss: 0.0093 - val_loss: 0.3530
Epoch 3/10
639/639 [==============================] - 86s 134ms/step - loss: 0.0079 - val_loss: 0.2590
Epoch 4/10
639/639 [==============================] - 85s 134ms/step - loss: 0.0065 - val_loss: 0.2309
Epoch 5/10
639/639 [==============================] - 86s 135ms/step - loss: 0.0065 - val_loss: 0.2236
Epoch 6/10
639/639 [==============================] - 87s 136ms/step - loss: 0.0048 - val_loss: 0.1946
Epoch 7/10
639/639 [==============================] - 86s 135ms/step - loss: 0.0049 - val_loss: 0.1188
Epoch 8/10
639/639 [==============================] - 87s 136ms/step - loss: 0.0046 - val_loss: 0.1337
Epoch 9/10
639/639 [==============================] - 85s 134ms/step - loss: 0.0046 - val_loss: 0.1412
Epoch 10/10
639/639 [==============================] - 85s 133ms/step - l

In [98]:
# Load the best weights
model.load_weights("best_weights.h5")

In [99]:
pred = model.predict(X_test)

In [100]:
Predictions = Target_Scaler.inverse_transform(pred)
Actual = Target_Scaler.inverse_transform(y_test)

In [101]:
Predictions.shape

(225, 1)

In [102]:
Predictions = np.squeeze(Predictions, axis=1)
Actual = np.squeeze(Actual, axis=1)

In [104]:
# plot Actual vs Predicted Values on test data
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:], y = Actual, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:], y = Predictions, mode='lines', name='Predictions'))
fig.show()

## Visualize Prediction on whole data

In [105]:
Total_features = np.concatenate((X_train, X_test), axis=0)
Total_Targets = np.concatenate((y_train, y_test), axis=0)

In [106]:
pred = model.predict(Total_features)

In [107]:
Predictions = Target_Scaler.inverse_transform(pred)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [108]:
Predictions = np.squeeze(Predictions, axis=1)
Actual = np.squeeze(Actual, axis=1)

In [110]:
# check the trend in volume traded
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=Actual, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=data.index, y=Predictions, mode='lines', name='Predictions'))
fig.show()

In [111]:
# save and load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

## Real Prediction

In [116]:
def PredictStockPrice(Model, DataFrame, PreviousDate, feature_length=32):
  idx_location = DataFrame.index.get_loc(PreviousDate)
  Features = DataFrame.iloc[idx_location - feature_length: idx_location, :].values
  Features = np.expand_dims(Features, axis=0)
  Features = Feature_Scaler.transform(Features)
  Prediction = Model.predict(Features)
  Prediction = Target_Scaler.inverse_transform(Prediction)
  return Prediction[0][0]

In [124]:
PredictStockPrice(loaded_model, data, '2021-07-23')

2142.63